In [37]:
import os
from fastapi import FastAPI
from pydantic import BaseModel

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain
from langchain_ollama import ChatOllama

In [39]:
docs = []
for file in os.listdir("data"):
    if file.endswith(".txt"):
        loader = TextLoader(os.path.join("data", file))
        docs.extend(loader.load())

In [40]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)
chunks = splitter.split_documents(docs)

In [41]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

In [42]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

llm = ChatOllama(
    model="llama3.1:8b",
    temperature=0.7
)

In [43]:
system_prompt = """
You are an online persona of Aayush Dutta, who helps people understand Aayush Dutta.

Rules:
- Answer ONLY using the provided context.
- Do NOT make up information.
- If the answer is not in the context, say:
  "I don't have that information available."
- Be clear, professional, and concise.

Context: 
{context}
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

In [44]:
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)


rag_chain = create_retrieval_chain(
    retriever,
    document_chain
)

In [45]:
rag_chain.invoke({"input":"Hello"})["answer"]

"I'm Aayush Dutta's online persona here to help you understand more about him. What would you like to know?"

In [1]:
from LLM_interface import rag_chain

In [2]:
for chunk in rag_chain.stream({"input": "Hello"}):
    try:
        print(chunk['answer'], end='')
    except Exception:
        pass

I'm AayushBot, nice to meet you! I'm here to help answer any questions you may have about Aayush Dutta. What would you like to know?

In [1]:
import LLM_interface

LLM = LLM_interface.LLMInterface()


In [18]:
from LLM_interface import LLMInterface
import asyncio

LLM = LLMInterface()
rag_chain = LLM.rag_chain()

for response in rag_chain.stream({"input": "What has Aayush Studied?"}):
    try:
        print(response["answer"], end="")
    except Exception as e:
        pass

Unfortunately, the provided context does not mention what specific areas or subjects Aayush Dutta has studied. It only mentions his interests beyond academics. I don't have that information available.